In this section we'll look at how to open a map, find a place on it, and plan a route. The required data for finding places and the calculations for the route are not done on our own computer but on remote servers. The communication with these remote servers are done using a **application programming interface** or **API**. There exist APIs for many different services such as checking the news or weather, or booking a hotel room. 

In this notebook we'll make use of APIs from [Open Street Maps](https://www.openstreetmap.org/)

We'll start by loading an interactive map using the 'folium' package:

In [1]:
import folium
folium.Map( 
        location=[52.0078168,4.3693158], # geo coordinates of center
        zoom_start=14.5,
        )

## Finding a place
The `geopy` package has tools to work with services that allow you to find locations (latitude, longitude, address). The `Nominatim` service is provided by Open Street Maps and is free, but limits the number of requests you can do. 

In [2]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="ds_sem1", timeout=2)

We can search for a place using the function `geocode`:

In [3]:
geo_hhs = geolocator.geocode('Haagse Hogeschool Delft') 

If the place is found, the variable `geo_hhs` has its address and location (longitude+latitude).

In [4]:
geo_hhs.address

'Haagse Hogeschool, 137, Rotterdamseweg, Zeeheldenbuurt, Wippolder, Delft, Zuid-Holland, Nederland, 2628 AL, Nederland'

In [5]:
geo_hhs[1]

(52.00161955, 4.368017768665558)

#### Find the geo data of the (central) station of Delft, and assign it to variable `geo_station`

In [6]:
%%assignment
### BEGIN SOLUTION
geo_station = geolocator.geocode('Station Delft')
### END SOLUTION
geo_station # leave this to print the result

Location(Station Delft, Barbarasteeg, Binnenstad, Delft, Zuid-Holland, Nederland, 2611 BM, Nederland, (52.0073979, 4.3581707, 0.0))

In [7]:
%%check
hashresult==2426117906

Correct!


We can add to markers of the found locations to the map using the longitude and latitude:

In [8]:
map = folium.Map(location=geo_hhs[1], zoom_start=16)
marker_hhs = folium.Marker( geo_hhs[1], tooltip='Haagse Hogeschool Delft')
marker_hhs.add_to(map)
map

## Planning a route
The `openrouteservice` package enables us to plan a route using Open Street Maps if you have a free account. We connect to the service using a key that is tied to our account:

In [9]:
from openrouteservice import client

api_key = '5b3ce3597851110001cf6248320933ccd0094bbb97277e2649618c4d'
ors = client.Client(key=api_key)

Now we can plan a route using the function `directions`. The locations for the route are specified in `coordinates` as a list of latitude/longitude pairs, which is annoyingly the reverse of the output of our geocoder.

In [10]:
route = ors.directions(profile='foot-walking', format_out='geojson', 
                        coordinates=[(g[1],g[0]) for g in (geo_hhs[1],geo_station[1])])
route


{'type': 'FeatureCollection',
 'features': [{'bbox': [4.358222, 52.000901, 4.368471, 52.007908],
   'type': 'Feature',
   'properties': {'segments': [{'distance': 1608.8,
      'duration': 1158.3,
      'steps': [{'distance': 253.4,
        'duration': 182.5,
        'type': 11,
        'instruction': 'Head northeast',
        'name': '-',
        'way_points': [0, 5]},
       {'distance': 18.7,
        'duration': 13.5,
        'type': 13,
        'instruction': 'Keep right',
        'name': '-',
        'way_points': [5, 10]},
       {'distance': 4.0,
        'duration': 2.9,
        'type': 0,
        'instruction': 'Turn left onto Rotterdamseweg',
        'name': 'Rotterdamseweg',
        'way_points': [10, 11]},
       {'distance': 13.8,
        'duration': 9.9,
        'type': 1,
        'instruction': 'Turn right',
        'name': '-',
        'way_points': [11, 12]},
       {'distance': 145.3,
        'duration': 104.6,
        'type': 6,
        'instruction': 'Continue straig

The route contains a lot of data. The duration of the route in minutes is given by:

In [11]:
route['features'][0]['properties']['segments'][0]['duration'] / 60

19.305

We can show the route on the map:

In [12]:
folium.features.GeoJson(route).add_to(map)
map

The function `distance_matrix` allows us to determine the distance between multiple locations. We'll add a third location for the Station Delft Campus:

In [13]:
geo_station_campus = geolocator.geocode('Station Delft Campus')

locations_delft = [(g.longitude,g.latitude) for g in [geo_hhs,geo_station,geo_station_campus]]

distance_matrix=ors.distance_matrix(profile="foot-walking", locations=locations_delft)

The distance from each location to each location is given in seconds:

In [14]:
distance_matrix['durations']

[[0.0, 1158.3, 1517.31], [1158.3, 0.0, 1801.54], [1517.31, 1801.54, 0.0]]

Note that this is a list of lists. `distance_matrix['durations'][i][j]` has the duration of seconds from `locations_delft[i]` to `locations_delft[j]`.

#### What's the (rounded) distance in walking seconds from the Haagse Hogeschool to Station Delft Campus?

In [15]:
%%slider
0 2000

IntSlider(value=1557, max=2000)

In [16]:
%%check
hashresult==2303615734

Correct!
